In [ ]:
import pandas as pd
import chardet

#### Objetivo
Conseguir un dataframe el cual nos pueda decir que vehiculos hay matriculados en un cierto mes y año.

Para ello necesitamos conocer el parque de vehiculos exacto en un punto inicial y poder saber los cambios respecto a ese punto.

En neustro caso tenemos el fichero del parque de vehiculos "mat_2034.txt" y tenemos todos los tramites desde el dia de hoy hasta diciembre de 2014. Lo que podemos hacer es añadir una columna extra para Año de Matriculación, una para Mes de Matriculación, Año de Baja y Mes de Baja. De manera que si seleccionamos los vehciulos que estan matriculados, por ejemplo, en Abril de 2021 tenemos que seleccionar todos los vehciulos con fecha de baja posterior a abril de 2021 (o sin fecha de baja) y una baja de matriculacion anterior o igual a abril de 2021.

#### Worflow
1. Priemro leemos el fichero de parque de vehciulos y eliminamos todas las columnas que no nso interean.
2. Comprobamos la fehca de matroculación más reciente para saber el punto de partida. (29-12-2023)
3. Tenemos que concatenar los diferentes DataFrames que tenemos sobre tramites de manera que:
    - Tengan las columnas que queremos
    - 4 columnas más: Año Baja | Mes Baja | Año Matr | Mes Matr, para conseguirlo tenemos que clasificar los diferentes tramites como Alta o Matriculacion y extraer la fecha en la que se dio dicho tramite
4. Una vez tenemos concatenados los dataframes de los tramites hay que concatenarlos con la foto exacta del parque

### 1. Leer el fichero

In [406]:
def reader(name, sep=',', columns= ['EMISIONES_CO2','FECHA_MATR']):
    chunksize = 10**4
    chunkreader = pd.read_csv(name, sep=sep, usecols=columns,
                              chunksize=chunksize, on_bad_lines='skip', low_memory=False) 
    #If bad lines not skipped we get an error bc of different types of data in the same columns
    #Columns (0,1,8,11,12,15,19,20,23,24,25,26,27,30,32,33,37,39,40,41,42,43,44) 

    for chunk in chunkreader:
        yield chunk

### 2. Comprobar la fecha más reciente

In [ ]:
fechas = []

for chunk in reader('mat_2023.txt',sep='|'):
    chunk['FECHA_MATR'] = pd.to_datetime(chunk['FECHA_MATR'], errors='coerce', dayfirst=True)
    chunk['MONTH_MAT'] = chunk['FECHA_MATR'].dt.month
    chunk['YEAR_MAT'] = chunk['FECHA_MATR'].dt.year
    fechas.append(chunk['FECHA_MATR'].max())

dates = pd.DataFrame({'NEWEST_MATR' : fechas})
newest_matr = dates['NEWEST_MATR'].max()
print(f'La matriculacion más reciente en la foto exacta del parque es del dia {newest_matr}')

2023-12-29 00:00:00


### 3. Concatenar los dataframes de tramites

Menuda broma como guardan los ficheros de tramites mensuales.
- Estan codificados con ISO-8859-1
- El fichero no tiene header, nombre de columnas definido en el codigo
- Es un fixed-width file y cada columna ocupa una cantidad diferente, definido en el codigo

Creo que FEC_TRAMITE (y/o FEC_TRAMITACION) indica cuando se hizo el tramite y FEC_PROCESO cuando ese tramite se hizo efectivo. Voy a coger FEC_TRAMITE por ahora

In [ ]:
with open('export_mensual_bajas_202501.txt', 'rb') as f:
    result = chardet.detect(f.read(11700))

encoding = result['encoding']
print(f'The encoding of the file is: {encoding}')

col_widths = [8,1,8,30,22,1,21,2,1,5,6,6,6,3,2,2,2,2,24,2,2,1,8,5,8,1,1,9,3,5,30,7,3,5,1,1,1,1,1,1,11,25,25,35,70,6,6,4,4,3,8,4,4,4,6,30,50,35,25,35,4,4,4,1,25,1,4,25,8]
col_names = ["FEC_MATRICULA","COD_CLASE_MAT","FEC_TRAMITACION","MARCA_ITV","MODELO_ITV","COD_PROCEDENCIA_ITV","BASTIDOR_ITV","COD_TIPO",
    "COD_PROPULSION_ITV","CILINDRADA_ITV","POTENCIA_ITV","TARA","PESO_MAX","NUM_PLAZAS","IND_PRECINTO","IND_EMBARGO","NUM_TRANSMISIONES",
    "NUM_TITULARES","LOCALIDAD_VEHICULO","COD_PROVINCIA_VEH","COD_PROVINCIA_MAT","CLAVE_TRAMITE","FEC_TRAMITE","CODIGO_POSTAL","FEC_PRIM_MATRICULACION",
    "IND_NUEVO_USADO","PERSONA_FISICA_JURIDICA","CODIGO_ITV","SERVICIO","COD_MUNICIPIO_INE_VEH","MUNICIPIO","KW_ITV","NUM_PLAZAS_MAX","CO2_ITV","RENTING", 
    "COD_TUTELA", "COD_POSESION","IND_BAJA_DEF","IND_BAJA_TEMP", "IND_SUSTRACCION","BAJA_TELEMATICA","TIPO_ITV","VARIANTE_ITV","VERSION_ITV",
    "FABRICANTE_ITV", "MASA_ORDEN_MARCHA_ITV","MASA_MAXIMA_TECNICA_ADMISIBLE_ITV","CATEGORÍA_HOMOLOGACION_EUROPEA_ITV","CARROCERIA","PLAZAS_PIE",
    "NIVEL_EMISIONES_EURO_ITV","CONSUMO_WH/KM_ITV","CLASIFICACIÓN_REGLAMENTO_VEHICULOS_ITV", "CATEGORÍA_VEHICULO_ELECTRICO","AUTONOMÍA_VEHÍCULO_ELÉCTRICO",
    "MARCA_VEHÍCULO_BASE","FABRICANTE_VEHÍCULO_BASE","TIPO_VEHÍCULO_BASE","VARIANTE_VEHÍCULO_BASE","VERSIÓN_VEHÍCULO_BASE","DISTANCIA_EJES_12_ITV",
    "VIA_ANTERIOR_ITV","VIA_POSTERIOR_ITV","TIPO_ALIMENTACION_ITV","CONTRASEÑA_HOMOLOGACION_ITV","ECO_INNOVACION_ITV","REDUCCION_ECO_ITV","CODIGO_ECO_ITV","FEC_PROCESO"
]

col_to_keep = ['CO2_ITV','FEC_MATRICULA','CLAVE_TRAMITE','FEC_TRAMITE',]

def tramit_reader(file_path,col_widths=col_widths,col_names=col_names, encoding='ISO-8859-1'):
    df = pd.read_fwf(
        file_path,
        widths=col_widths,
        encoding=encoding,
        header=None,
    )
    df.columns = col_names
    return df

def normalize_date(date_str):
    date_str = str(date_str).strip()
    if len(date_str) == 7:
        return '0' + date_str
    return date_str

#Tengo dudas si el 5 deberia contar como matriculacion
def tramit_checker(df):
    Mat = [1,5,8,9]
    Baja = [0,3,4,6,7]

    clave_tramite = df['CLAVE_TRAMITE'][0]

    if clave_tramite in Baja:
        month = df['FEC_TRAMITE'].dt.month
        year = df['FEC_TRAMITE'].dt.year
        fecha_baja = pd.concat([month, year], axis=1)
        fecha_baja.columns = ['MONTH_BAJA','YEAR_BAJA']
        return fecha_baja
    else:
        return float("nan")
    
def tramit_proces(df):
    df['FEC_TRAMITE'] = df['FEC_TRAMITE'].apply(normalize_date)
    df['FEC_TRAMITE'] = pd.to_datetime(df['FEC_TRAMITE'], format='%d%m%Y')
    df['FEC_MATRICULA'] = df['FEC_MATRICULA'].apply(normalize_date)
    df['FEC_MATRICULA'] = pd.to_datetime(df['FEC_MATRICULA'], format='%d%m%Y')
    df['MONTH_MAT'] = df['FEC_MATRICULA'].dt.month
    df['YEAR_MAT'] = df['FEC_MATRICULA'].dt.year
    df = pd.concat([df, tramit_checker(df)], axis=1)
    df = df.drop(['FEC_MATRICULA','CLAVE_TRAMITE','FEC_TRAMITE'], axis=1)
    return df

In [ ]:
files = ['export_mensual_bajas_202412.txt','export_mensual_bajas_202501.txt','export_mensual_bajas_202502.txt']
col_to_keep = ['CO2_ITV','FEC_MATRICULA','CLAVE_TRAMITE','FEC_TRAMITE',]

for x,file in enumerate(files):
    if x == 0:
        df = tramit_reader(file)
        df = df[col_to_keep]
        procesado = tramit_proces(df)

    else:
        df = tramit_reader(file)
        df = df[col_to_keep]
        procesado = pd.concat([procesado, tramit_proces(df)], axis=0)

1 : Matrciulacion ordinaria y de ciclomotores : Mat
2 : Transferencia : ---
3 : baja definitiva (exclusion Plan Renove, Baja por exportación y Transito comunitario) : Baja
4 : Baja definitica por Plan Renove : Baja
5 : Rematriculación : Mat
6 : Baja temporal : Baja
7 : Baja definitiva por Exportación y por Transito comunitario : Baja
8 : Matriculacion vehiculo especial : Mat
9 : Matriculacion temporal : Mat
A : Prorroga matricula temporal ---
B : Paso de matricula temporal a dfinitiva : --- 

In [ ]:
for i in range(0,len(chunks)):
    chunk = chunks[i]
    nans = chunk[chunk['EMISIONES_CO2'].isna()]
    values = chunk[chunk['EMISIONES_CO2'].notna()]

    nans.to_csv('nans.csv', mode='a', index=False)
    values.to_csv('values.csv', mode='a', index=False)

In [16]:
def reader(name, sep):
    chunksize = 10**5
    chunkreader = pd.read_csv(name, sep=sep, chunksize=chunksize, on_bad_lines='skip') 
    #If bad lines not skipped we get an error bc of different types of data in the same columns
    #Columns (0,1,8,11,12,15,19,20,23,24,25,26,27,30,32,33,37,39,40,41,42,43,44) 

    for chunk in chunkreader:
        yield chunk

In [18]:
first_chunk = True

for chunk in (reader('mat_2023.txt', sep="|")):
    nans = chunk[chunk['EMISIONES_CO2'].isna()]
    values = chunk[chunk['EMISIONES_CO2'].notna()]

    nans.to_csv('Nans.csv', mode='a', index=False, header=first_chunk)
    values.to_csv('Values.csv', mode='a', index=False, header=first_chunk)

    first_chunk = False


/var/folders/gk/9s2n71rj4fq128zrj3x599pr0000gn/T/ipykernel_5549/3943720015.py:7: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunkreader:
/var/folders/gk/9s2n71rj4fq128zrj3x599pr0000gn/T/ipykernel_5549/3943720015.py:7: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunkreader:
/var/folders/gk/9s2n71rj4fq128zrj3x599pr0000gn/T/ipykernel_5549/3943720015.py:7: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunkreader:
/var/folders/gk/9s2n71rj4fq128zrj3x599pr0000gn/T/ipykernel_5549/3943720015.py:7: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunkreader:
/var/folders/gk/9s2n71rj4fq128zrj3x599pr0000gn/T/ipykernel_5549/3943720015.py:7: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or 

KeyboardInterrupt: 